In [11]:
from __future__ import print_function, division
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

firefoxdriver = "/usr/local/bin/geckodriver" # path to the chromedriver executable
os.environ["webdriver.firefox.driver"] = firefoxdriver


%config InlineBackend.figure_format = 'png' # ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’

%matplotlib inline
mpl.rcParams['figure.dpi']= 300

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

In [12]:
url = 'https://www.wta.org/go-hiking/hikes/wilderness-peak'
pacurl = 'https://www.wta.org/go-hiking/hikes/pacific-northwest-trail'
response = requests.get(url)
response2 = requests.get(pacurl)

page = response.text
page2 = response2.text

In [13]:
pac = BeautifulSoup(page2, 'lxml')

In [14]:
soup = BeautifulSoup(page, 'lxml')

In [15]:
headers = [
    'name',
    'region',
    'subregion',
    'votes',
    'rating',
    'length',
    'lengthtype',
    'gain',
    'hpoint',
    'fee',
    'lat',
    'long',
    'trailhead1',
    'trailhead2',
    'author1',
    'author2',
    'countreports',
          ]

binary_headers = [
    'Wildflowers/Meadows',
    'Ridges/passes',
    'Wildlife',
    'Waterfalls',
    'Old growth',
    'Summits',
    'Good for kids',
    'Dogs allowed on leash',
    'Fall foliage',
    'Lakes',
    'Rivers',
    'Coast',
    'Mountain views',
    'Established campsites',
]

# Helper Functions

In [16]:
def get_soup_hike_stat(soup, stat):
    obj = soup.find(text=re.compile(stat))
    if not obj:
        return None
    else:
        return obj

# Retrieval Functions

In [17]:
def get_name(soup):
    return soup.find(class_='documentFirstHeading').text

def get_region(soup):
    regions = get_soup_hike_stat(soup, 'Location').find_next().text
    regions = regions.split(sep="--")
    return regions[0].strip()

def get_subregion(soup):
    regions = get_soup_hike_stat(soup, 'Location').find_next().text
    regions = regions.split(sep="--")
    return regions[1].strip()

def get_length(soup):
    length = get_soup_hike_stat(soup, 'Length').find_next().find('span').text
    return length.split()[0]

def get_lengthtype(soup):
    length = get_soup_hike_stat(soup, 'Length').find_next().find('span').text
    return length.split()[2]

def get_gain(soup):
    return get_soup_hike_stat(soup, 'Gain').find_next().text

def get_hpoint(soup):
    return get_soup_hike_stat(soup, 'Highest Point').find_next().text

def get_fee(soup):
    return get_soup_hike_stat(soup, 'Parking Pass/Entry Fee').find_next().text

def get_lat(soup):
    return get_soup_hike_stat(soup, 'Co-ordinates').find_next().text

def get_long(soup):    
    return get_soup_hike_stat(soup, 'Co-ordinates').find_next().find_next().text

def get_trailhead1(soup):
    obj = soup.find(id='trailhead-details').find_all('p')[1].text
    if "weather" in obj:
        return np.nan
    return obj
def get_trailhead2(soup):
    obj = soup.find(id='trailhead-details').find_all('p')[2].text
    if "weather" in obj:
        return np.nan
    return obj

def get_author1(soup):
    return soup.find(class_='authorship sidebar-section').find('p').find_all('span')[0].text

def get_author2(soup):
    return soup.find(class_='authorship sidebar-section').find('p').find_all('span')[1].text

def get_countreports(soup):
    return soup.find(class_='ReportCount').text

def get_votes(soup):
    return soup.find(class_='rating-count').text.split()[0][1:]

def get_rating(soup):
    return soup.find(class_='current-rating').text.split()[0]

# Function to mark if certain categorical features are present

## Only run this after having created the dictionary for the current webpage

In [18]:
def append_features(soup, headers):
    '''
    soup: beautifulsoup object of web page
    headers: list of names of the particular wta.org trail page features we want.
    ----
    output is defaultdict
    '''
    feat_dict = defaultdict()
    for i in headers:
        try:
            # allows to call methods by inserting a string into the func name
            feat_dict[i] = [globals()["get_" + i](soup)]
        except:
            feat_dict[i] = np.nan
    return feat_dict

def append_binary_feats(soup, binary_headers, feat_dict):
    '''
    soup: beautifulsoup object of web page
    datadict: this is the dictionary used for each webpage to store the variables.
    Thus this function should be called only after that dict has been created.
    ----
    Product: this outcome appends binary categorical features present as a "1" or "0" to an existing
    dictionary
    '''
    bin_features = binary_headers.copy()
    for i in soup.find_all(class_='feature'):
        label = i.attrs['data-title']
        if label in bin_features:
            feat_dict[label] = 1
            bin_features.remove(label)
    for remainder in bin_features:
        feat_dict[remainder] = 0
    return feat_dict

# Creating Dictionary

# Combine into one scrape function

In [22]:
def scrape_page(soup, headers=headers, binary_headers=binary_headers):
    feat_dict = append_features(soup, headers)
    feat_dict = append_binary_feats(soup, binary_headers, feat_dict)
    # Create pandas DataFrame from dictionary
    df_page = pd.DataFrame.from_dict(feat_dict)
    return df_page

In [23]:
def concat_to_master(df_page, old_df=None):
    if old_df is not None:
        return pd.concat([old_df, df_page], join='inner')
    else:
        print("only 1 DataFrame present - returning it back to you")
        return df_page

# Beautiful Soup and Scraping

In [66]:
new_page_df = None
masterdf = None

In [ ]:
# loop through url from 0 to 3540, in increments of 30
# End of URL has simple integer that counts upward by 30 on each "Next 30" page of list of trails
# https://www.wta.org/go-outside/hikes/?b_start:int=0

Must substantiate first row of df

### This is it! The following cell creates the WTA df with all the data.

### **WARNING!!** Cell will take ~ 1 hour and 9 minutes to execute.

In [83]:
def scrape_and_create_all_wta():
    '''
    Function scrapes every WTA hike page, concatenates info, and returns Pandas DF.
    WARNING!!! This function takes approximately 1 hours and 9 minutes to run
    '''
    # tripreports url changes by number at end. iterates by 30 - begins at 0, ends at 3540
    df = None
    for i in range(0, 3541, 30): 
        wtaurl = f'https://www.wta.org/go-outside/hikes/?b_start:int={i}'
        wtaresponse = requests.get(wtaurl)
        wta_html = wtaresponse.text
        wta_soup = BeautifulSoup(wta_html, 'lxml')
        for j in wta_soup.find_all(class_='listitem-title'):
            page_url = j['href']
            page_response = requests.get(page_url)
            page_html = page_response.text
            page_soup = BeautifulSoup(page_html, 'lxml')
            new_page_df = scrape_page(page_soup)
            df = concat_to_master(new_page_df, df)
    return df

In [75]:
df = scrape_and_create_all_wta()

only 1 DataFrame present - returning it back to you


(3555, 31)

In [85]:
df.sample(5)

,name,region,subregion,votes,rating,length,lengthtype,gain,hpoint,fee,lat,long,trailhead1,trailhead2,author1,author2,countreports,Wildflowers/Meadows,Mountain views,Dogs allowed on leash,Established campsites,Waterfalls,Good for kids,Fall foliage,Ridges/passes,Wildlife,Old growth,Summits,Lakes,Rivers,Coast
0,Bears Breast,Central Cascades,NaN,3,3.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Keenes Trail,South Cascades,NaN,4,3.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Keenes Trail (#120),NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,Olallie Creek Snowshoe,Mount Rainier Area,SE - Cayuse Pass/Steven's Canyon,2,2.00,6.8,roundtrip,1800,3800,NaN,NaN,NaN,Mount Rainier National Park,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
0,Clackamas Mountain,Eastern Washington,Okanogan Highlands/Kettle River Range,3,4.33,10.0,roundtrip,NaN,NaN,NaN,48.6805,-118.9069,Clackamas Mountain (#301),"Colville National Forest, Republic Ranger Dist...",WTA Community,Multiple authors contributed to this report,4,0,1,1,0,0,0,0,0,1,0,0,0,0,0
0,Trap Lake,Central Cascades,Stevens Pass - West,4,5.00,11.0,roundtrip,2700,5400,None,47.7185,-121.1099,"Mt. Baker-Snoqualmie National Forest, Skykomis...",NaN,WTA Correspondents,Charles Ardary,43,1,1,1,0,0,0,0,0,0,1,0,1,0,0


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3555 entries, 0 to 0
Data columns (total 31 columns):
name                     3555 non-null object
region                   3555 non-null object
subregion                2942 non-null object
votes                    3555 non-null object
rating                   3555 non-null object
length                   2361 non-null object
lengthtype               2219 non-null object
gain                     1854 non-null object
hpoint                   1946 non-null object
fee                      2013 non-null object
lat                      2607 non-null object
long                     2607 non-null object
trailhead1               2961 non-null object
trailhead2               1081 non-null object
author1                  1291 non-null object
author2                  1291 non-null object
countreports             3555 non-null object
Wildflowers/Meadows      3555 non-null int64
Mountain views           3555 non-null int64
Dogs allowed on leash   